In [ ]:
from ligo.skymap.io import read_sky_map
import astropy_healpix as ah
import numpy as np
from ligo.gracedb.rest import GraceDb
import wget
from astropy.io import fits
import requests
import os
import healpy as hp
from astropy.coordinates import SkyCoord
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import crossmatch
from astropy import units as u
import astropy.cosmology.units as cu
from astropy.cosmology import WMAP9
from astropy.table import Table, vstack,unique
from astropy import table
from astropy.io import ascii
import pandas as pd
import mastcasjobs
import time
from io import StringIO

In [ ]:
def area_90(graceid):
    # input a list of GraceIDs and return the area of the 90% probability region and other information for each event and the mean,std of the distance

    client = GraceDb()
    areas = np.zeros(len(graceid))
    distmean = np.zeros(len(graceid))
    diststd = np.zeros(len(graceid))
    ras = np.zeros(len(graceid))
    decs = np.zeros(len(graceid))
    prob_dec_gt_m30 = np.zeros(len(graceid))

    for j in range(len(graceid)):
        name = graceid[j]

        if os.path.exists('/data/GW_events/data/{}_multiorder.fits'.format(name)):
            # If the skymap file already exists, read it directly
            skymap = read_sky_map('/data/GW_events/data/{}_multiorder.fits'.format(name), distances=True, nest=True, moc=True)

        else: 
            #Get the event skymap
            response = client.superevent(name)
            file = response.json()['links']['files']

            suffixes = ['Bilby.multiorder.fits','Bilby.offline0.multiorder.fits','bayestar.multiorder.fits']
            url_found = False
            
            for suffix in suffixes:
                url = f"{file}{suffix}"
                try:
                    print(url)
                    response = requests.get(url)
                    if response.status_code == 200:
                        url_found = True
                        break
                except Exception as e:
                    print(f'Error fetching URL {url}: {e}')

            if not url_found:
                print(f'No skymap found for event {name} at index {j}')
                exit(0) # 跳过当前事件，继续处理下一个

            print(j, name, url)
            print(wget.download(url, out = '/data/GW_events/data/{}_multiorder.fits'.format(name)))
            skymap = read_sky_map('/data/GW_events/data/{}_multiorder.fits'.format(name), distances=True, nest=True, moc=True)
            


        # Find the 90% Probability Region
        skymap.sort('PROBDENSITY', reverse=True)
        level,ipix = ah.uniq_to_level_ipix(skymap['UNIQ'])
        nside = ah.level_to_nside(level)
        pixel_area = ah.nside_to_pixel_area(nside)
        prob = pixel_area * skymap['PROBDENSITY']
        cumprob = np.cumsum(prob)
        i = cumprob.searchsorted(0.9)
        area_90 = pixel_area[:i].sum()
        areas[j] = '{:.4f}'.format(area_90.to_value(u.deg**2))
        print(area_90.to_value(u.deg**2))

        # Find the most probable ra and dec
        i = np.argmax(skymap['PROBDENSITY'])
        uniq = skymap[i]['UNIQ']
        level, ipix = ah.uniq_to_level_ipix(uniq)
        nside = ah.level_to_nside(level)
        ra, dec = ah.healpix_to_lonlat(ipix, nside, order='nested')
        ras[j] = '{:.4f}'.format(ra.deg)
        decs[j] = '{:.4f}'.format(dec.deg)

        # cauluclate the probability of dec > -30
        level,ipix = ah.uniq_to_level_ipix(skymap['UNIQ'])
        nside = ah.level_to_nside(level)
        pixel_area = ah.nside_to_pixel_area(nside)
        prob = pixel_area * skymap['PROBDENSITY']
        lon = np.zeros(len(nside))
        lat = np.zeros(len(nside))
        for k in range(len(nside)):
            lon[k], lat[k] = hp.pix2ang(nside[k], ipix[k], nest=True, lonlat=True) 
        index = np.where(lat > -30)
        prob_dec_gt_m30[j] = '{:.4f}'.format(prob[index].sum())


        # get the mean and std of the distance
        hdul = fits.open("/data/GW_events/data/{}_multiorder.fits".format(name))
        hdr = hdul[1].header
        distmean[j] = '{:.4f}'.format(hdr['DISTMEAN'])
        diststd[j] = '{:.4f}'.format(hdr['DISTSTD'])
        hdul.close()

    return areas, distmean, diststd, ras, decs, prob_dec_gt_m30

In [ ]:
def crossmatch_mq(gracedbid, out_file):

    #get the milliquas catalogue
    # if don't have the milliquas catalogue, you can download it from Vizier as follows:
    # from astroquery.vizier import VizierClass
    # vizier = VizierClass(
    # row_limit=-1,
    # columns=['recno', 'RAJ2000', 'DEJ2000', 'z','Name','Type']
    # )
    # cat, = vizier.get_catalogs('VII/294/catalog')
    # cat.sort('recno')  # sort catalog so that doctest output is stable
    # del cat['recno']
    cat = ascii.read('/data/milliquas.csv', format='csv')
    # calculate the distance of each milliquas from z
    z = cat['z'] * cu.redshift
    dis = z.to(u.Mpc, cu.redshift_distance(WMAP9, kind="luminosity"))
    cat.add_column(dis,name='Distance')
    coordinates = SkyCoord(cat['RAJ2000']*u.deg, cat['DEJ2000']*u.deg,cat['Distance'])
    res = Table()   #the matched catalogue of all events
    match_num = np.zeros(len(gracedbid),dtype=int)  #the number of matched AGNs for each event

    for j in range(len(gracedbid)):
        #read skymap
        name = gracedbid[j]
        
        if os.path.exists('/data/GW_events/match_mq/{}.csv'.format(name)):
            match_tab = ascii.read('/data/GW_events/match_mq/{}.csv'.format(name), format='csv')
            match_num[j] = len(match_tab)
            if match_num[j] == 0:
                continue
        else:
            skymap = read_sky_map('/data/GW_events/data/{}_multiorder.fits'.format(name), distances=True, nest=True, moc=True)
            result = crossmatch(skymap, coordinates)
            match_tab = cat[result.searched_prob_vol < 0.9]  #the matched catalogue of each event
            match_tab.write('/data/GW_events/match_mq/{}.csv'.format(name), format='csv', overwrite=True)
            match_num[j] = len(match_tab)
        if j==0:
            res = match_tab
        else:
            res = vstack([res, match_tab], join_type='exact')
             
    #save the catlogue
    res = table.unique(res, keys='Name')
    res.write('/data/GW_events/{}.csv'.format(out_file), format='csv', overwrite=True)

    return match_num

In [ ]:
#read the events csv
event = 'O4b_events'
area = '500deg'
O4ab = event[0:3]
file_path = ""  #the path of the GW events csv file
events = ascii.read(file_path,format='csv')
graceid = events['superevent_id']
print(graceid)
areas, distmean, diststd, ra, dec, prob_dec_gt_m30 = area_90(graceid)
match_num = crossmatch_mq(graceid, out_file='/{}/{}/{}_withoutfilter_mq_all'.format(area, O4ab , event))
events.add_column(areas, name='area_90')
events.add_column(distmean, name='distmean')
events.add_column(diststd, name='diststd')
events.add_column(ra, name='ra')
events.add_column(dec, name='dec')
events.add_column(prob_dec_gt_m30, name='prob_dec_gt_m30')
events.add_column(match_num, name='match_num')
d = events['distmean'] * u.Mpc
z = d.to(cu.redshift, cu.redshift_distance(WMAP9, kind="comoving", zmax=1200))
events.add_column(z, name='zmean')

# save the events csv file without filter, with more detailed information
events.write('/data/GW_events/{}/{}/{}_without_filter.csv'.format(area,O4ab, event), format='csv', overwrite=True)
index = np.where(areas < float(area[0:3]))
events = events[index]
print(events)
print(len(events))
events.write('/data/GW_events/{}/{}/{}_filtered_{}.csv'.format(area,O4ab, event, area), format='csv', overwrite=True)

#crossmatch with mq catalog
if len(index)>0:
    crossmatch_mq(events['superevent_id'], out_file='/{}/{}/{}_{}_mq_matched'.format(area, O4ab, event,area))
else:
    print(f'No events have area_90 < {area}^2')

In [ ]:
# filter with declination > -30
mq = pd.read_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_mq_matched.csv')
print("All match AGNs:", len(mq))
dec_gtm30 = mq[mq['DEJ2000'] > -30]
dec_gtm30.to_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_mq_matched_dec_gtm30.csv', index=False)
print("Matched AGNs with declination > -30:", len(dec_gtm30))

In [ ]:
# Use MAST CasJobs to filter events by PS1 g/r mag
from astropy.table import join
# MAST CasJobs credentials
user = "supernova"
pwd = "sbs1rqlb6x"

jobs = mastcasjobs.MastCasJobs(username=user, password=pwd, context="PanSTARRS_DR2",request_type='POST')

# get the table of events
events = ascii.read(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_mq_matched.csv',format='csv')
events_withoutname = events.copy()
events_withoutname.remove_column('Name')
jobs.drop_table_if_exists(f'{event}_{area}_mq_matched_PS1')
jobs.drop_table_if_exists(f'{event}_{area}_mq_matched')
jobs.upload_table(f'{event}_{area}_mq_matched', events_withoutname, verbose=True)


query=f"""SELECT d.RAJ2000, d.DEJ2000, d.z, d.Type, d.Distance,
o.objID, o.gMeanKronMag, o.rMeanKronMag,o.nDetections,
soa.primaryDetection
 INTO mydb.[{event}_{area}_mq_matched_PS1]
 FROM mydb.[{event}_{area}_mq_matched] d
CROSS APPLY dbo.fGetNearbyObjEq(d.RAJ2000, d.DEJ2000, 3.0/60.0) as x
JOIN MeanObjectView o on o.ObjID=x.ObjId
LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
WHERE o.nDetections>5
AND soa.primaryDetection>0
AND o.gMeanKronMag < 21
AND o.rMeanKronMag < 21
AND o.gMeanKronMag > 0
AND o.rMeanKronMag > 0
AND o.decMean > -30
"""
print(query)
jobid = jobs.submit(query, task_name='filter with g/r mag')
print(jobid)
code,status = jobs.status(jobid)   #status: ready, started, finished, failed;code:0,1,5
while status not in ['finished', 'failed']:
    print(code, status)
    time.sleep(5)  # 等待10秒后再次检查状态
    code,status = jobs.status(jobid)
if status == 'failed':
    raise Exception("Job failed to complete.")

events_filtered = jobs.fast_table(f'{event}_{area}_mq_matched_PS1', verbose=True)
print(len(events_filtered))

events_filtered = join(events_filtered,events,join_type='left',keys=['z','Type','Distance'])
events_filtered.remove_columns(['RAJ2000_1',"DEJ2000_1"])
events_filtered.rename_column('RAJ2000_2','RAJ2000')
events_filtered.rename_column('DEJ2000_2','DEJ2000')

events_filtered.write(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_mq_matched_PS1.csv', format='csv', overwrite=True)

In [ ]:
def PS1_API_crossmatch(ra:np.array, dec:np.array, uploadCSV_path:str="./"):
    match_data = pd.DataFrame({"ra":ra, "dec":dec})
    match_length = len(ra)
    single_match_num = 4900
    split_idx = list(range(0,match_length,single_match_num))
    split_idx.append(match_length)
    
    # 使用API做crossmatch匹配半径只能为3角秒, When using API for crossmatch, the search radius can only be 3 arcseconds.
    DR = "dr2" # data release
    url = f'https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/{DR}/mean/crossmatch/upload?radius=0.0002'
    if match_length > single_match_num and len(split_idx) > 2:
        print("(PS1 crossmatch:Number of matches is too large,%i,Split it automatically" %single_match_num)
    results = []  
    uploadCSV_file = os.path.join(uploadCSV_path, "upload.csv")
    for i in range(len(split_idx)-1):
        idx_star = split_idx[i]
        idx_end = split_idx[i+1]
        match_data[idx_star:idx_end].to_csv(uploadCSV_file, index=True, index_label="source_idx")
        r = requests.post(url, files=dict(file=open(uploadCSV_file,'rb')))
        PS1_result = pd.read_csv(StringIO(r.text))
        try:
            PS1_result["_searchID_"] = PS1_result["_searchID_"] + i*(single_match_num) - 2
        except:
            print(PS1_result)
        results.append(PS1_result) 
    results = pd.concat(results)
    return results

In [ ]:
# Use API to crossmatch GW events with PS1

agns = pd.read_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_mq_matched_dec_gtm30.csv')
print(len(agns))
ra = agns.iloc[:]['RAJ2000']
dec = agns.iloc[:]['DEJ2000']
match_results = PS1_API_crossmatch(ra, dec,f'/data/GW_events/{area}/{O4ab}/')
match_results.to_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_PS1_match_raw.csv',index=False)
# match_results = pd.read_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_PS1_match_raw.csv')
print(len(match_results))

# filter the results by nDetections, gMeanKronMag, rMeanKronMag
match_results = match_results.iloc[np.where(match_results['nDetections']>=5)]
match_results = match_results.iloc[np.where(np.logical_and(match_results['gMeanKronMag']<21,match_results['rMeanKronMag']<21))]
match_results = match_results.iloc[np.where(np.logical_and(match_results['gMeanKronMag']>0,match_results['rMeanKronMag']>0))]
print(len(match_results))
match_agns = pd.DataFrame(columns=['_ra_','_dec_'])

# ulitilize the dstArcSec to select the closest match
for i in range(len(agns)):
    idx = np.where(np.logical_and(match_results['_ra_']==agns.iloc[i]['RAJ2000'],match_results['_dec_']==agns.iloc[i]['DEJ2000']))
    if len(idx)==0:
        continue
    match_agn = match_results.iloc[idx]
    match_agn = match_agn.iloc[np.where(match_agn['dstArcSec']==np.min(match_agn['dstArcSec']))]
    if i == 0:
        match_agns = match_agn
    else:
        match_agns = pd.concat([match_agns,match_agn])

# JOIN query results with agns table and save the final results
match_agns = match_agns.iloc[:,[0,1,42,55,71]]
match_agns.columns = ['RAJ2000','DEJ2000','nDetections','gMeanKronMag','rMeanKronMag']
results = pd.merge(agns,match_agns,on=['RAJ2000','DEJ2000'],how='right')
print(len(results))
results.drop_duplicates(subset=['Name'],inplace=True)
results.to_csv(f'/data/GW_events/{area}/{O4ab}/{event}_{area}_PS1_match.csv',index=False)